<a href="https://colab.research.google.com/github/asifmoeed/Fire-Smoke-detection/blob/main/Fire_Project_10052025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2
import numpy as np
import threading
import time
from twilio.rest import Client
import firebase_admin
from firebase_admin import credentials, messaging
from ultralytics import YOLO
import requests


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
!pip install twilio firebase-admin ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
  

In [4]:
from ultralytics import YOLO

In [5]:
# Load YOLOv8 model (official Ultralytics implementation)
model = YOLO("yolov8n.pt")  # Nano version (fastest)

100%|██████████| 6.25M/6.25M [00:00<00:00, 87.2MB/s]


In [6]:
CLASS_MAP = {
    0: "person",
    # These are common class IDs for fire-related objects in custom-trained models
    # You'll need to train your own model or use these placeholder IDs
    25: "fire",
    26: "smoke",
    27: "flames"
}

In [8]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.2 MB/s eta 0:00:00


In [9]:
import cv2
import streamlit as st
from ultralytics import YOLO
import numpy as np
from PIL import Image
import tempfile

# --- Streamlit UI setup ---
st.set_page_config(page_title="Fire & Smoke Detection", page_icon="🔥", layout="wide")
st.title("🔥 Fire & Smoke Detection System")

# Sidebar settings
input_source = st.sidebar.radio("Select Input Source:", ("Webcam", "Mobile/IP Camera", "Upload Image", "Upload Video"))
conf_threshold = st.sidebar.slider("Detection Confidence Threshold", 0.1, 1.0, 0.5, 0.01)

# Load YOLO model
@st.cache_resource
def load_model():
    return YOLO("yolov8n.pt")  # Replace with "best.pt" if using your own trained model

model = load_model()

# Placeholder for messages and video
video_placeholder = st.empty()
status_text = st.empty()

def detect_and_display(frame, conf=0.5):
    results = model(frame, conf=conf)
    annotated = results[0].plot()
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    fire_detected = any(int(box.cls) in [25, 26, 27] for box in results[0].boxes)
    return annotated_rgb, fire_detected

# Handle input options
if input_source == "Webcam":
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        st.error("Webcam not accessible")
    else:
        st.info("Press 'Stop' to end the webcam feed")
        stop_button = st.button("Stop")
        while not stop_button:
            ret, frame = cap.read()
            if not ret:
                status_text.warning("Failed to read webcam feed")
                break
            annotated_frame, fire_detected = detect_and_display(frame, conf=conf_threshold)
            video_placeholder.image(annotated_frame, channels="RGB", use_column_width=True)
            if fire_detected:
                status_text.error("🚨 FIRE DETECTED!")
            else:
                status_text.success("✅ No fire detected")
            stop_button = st.button("Stop")
        cap.release()

elif input_source == "Mobile/IP Camera":
    ip_url = st.sidebar.text_input("Enter Mobile/IP Camera URL (e.g., http://192.168.X.X:8080/video)")
    if ip_url:
        cap = cv2.VideoCapture(ip_url)
        if not cap.isOpened():
            st.error("Unable to access IP Camera")
        else:
            st.info("Press 'Stop' to end the IP camera feed")
            stop_button = st.button("Stop")
            while not stop_button:
                ret, frame = cap.read()
                if not ret:
                    status_text.warning("Failed to read IP camera feed")
                    break
                annotated_frame, fire_detected = detect_and_display(frame, conf=conf_threshold)
                video_placeholder.image(annotated_frame, channels="RGB", use_column_width=True)
                if fire_detected:
                    status_text.error("🚨 FIRE DETECTED!")
                else:
                    status_text.success("✅ No fire detected")
                stop_button = st.button("Stop")
            cap.release()
    else:
        st.warning("Please enter a valid IP Camera URL")

elif input_source == "Upload Image":
    uploaded_img = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
    if uploaded_img is not None:
        img = Image.open(uploaded_img)
        frame = np.array(img)
        annotated_frame, fire_detected = detect_and_display(frame, conf=conf_threshold)
        st.image(annotated_frame, channels="RGB", use_column_width=True)
        if fire_detected:
            st.error("🚨 FIRE DETECTED!")
        else:
            st.success("✅ No fire detected")

elif input_source == "Upload Video":
    uploaded_vid = st.file_uploader("Upload a video", type=["mp4", "mov", "avi"])
    if uploaded_vid is not None:
        tfile = tempfile.NamedTemporaryFile(delete=False)
        tfile.write(uploaded_vid.read())
        cap = cv2.VideoCapture(tfile.name)
        st.info("Processing video...")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            annotated_frame, fire_detected = detect_and_display(frame, conf=conf_threshold)
            video_placeholder.image(annotated_frame, channels="RGB", use_column_width=True)
            if fire_detected:
                status_text.error("🚨 FIRE DETECTED!")
            else:
                status_text.success("✅ No fire detected")
        cap.release()


2025-06-01 20:05:35.298 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-01 20:05:35.302 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-01 20:05:35.474 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-01 20:05:35.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-01 20:05:35.479 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-01 20:05:35.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-01 20:05:35.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [10]:
!pip install streamlit ultralytics opencv-python
